### Step 1
read Data

In [36]:
import pandas as pd

class DataSet:
    def __init__(self, X=None, Y=None, File=None, Vectorizer=None):
        self.X = X
        self.Y = Y
        self.File = File

train = DataSet(File="assets/adult_train.csv")
test = DataSet(File="assets/adult_test.csv")
datasets = [train, test]

for dset in datasets:
    csvdat = pd.read_csv(dset.File)
    data_shuffled = csvdat.sample(frac=1.0, random_state=0)
    
    dset.X = data_shuffled.drop('target', axis=1)
    dset.Y = data_shuffled['target']

In [34]:
# Train Preview
print(train.Y.head())
train.X.head()

22278    <=50K
8950     <=50K
7838     <=50K
16505    <=50K
19140     >50K
Name: target, dtype: object


,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
22278,49,Local-gov,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States
8950,49,Private,HS-grad,9,Divorced,Other-service,Not-in-family,Black,Female,0,0,40,United-States
7838,31,Private,Prof-school,15,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,50,United-States
16505,53,Private,Some-college,10,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,43,United-States
19140,30,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States


### Step 2
Dict Vectorizer

In [35]:
from sklearn.feature_extraction import DictVectorizer
 
xTrainRecords = train.X.to_dict("records")
vectorizer = DictVectorizer(sparse=False)
xTrainEncoded = vectorizer.fit_transform(xTrainRecords)

vectorizer.transform(test.X.to_dict("records"))

array([[44.,  0.,  0., ...,  0.,  0.,  0.],
       [37.,  0.,  0., ...,  0.,  0.,  0.],
       [35.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [54.,  0.,  0., ...,  1.,  0.,  0.],
       [44.,  0.,  0., ...,  0.,  0.,  0.],
       [59.,  0.,  0., ...,  0.,  1.,  0.]])

### Step 3
pipelining it

In [46]:
import sklearn as sk
from sklearn.pipeline import make_pipeline
import TreeClassifers
  
pipeline = sk.pipeline.make_pipeline(
  DictVectorizer(),
  TreeClassifier()
)

NameError: name 'TreeClassifier' is not defined